In [ ]:
#import pluto data
import pandas as pd
import numpy as np

dfPluto =  pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\PLUTO\\MapPLUTO_BINs_1toManyJOIN.csv", low_memory=False )


In [ ]:
#dfOwn = dfPluto.set_index('BIN')['OwnerName'].to_dict()
dfPluto["bin"] = dfPluto["bin"].astype(int)
dfPluto["bin"] = dfPluto["bin"].astype(str)
dfOwn = dfPluto.set_index('bin')['OwnerName'].to_dict()

In [ ]:
dfo = dfPluto[["bin", "OwnerName"]]

In [ ]:
dfo.head(3)

In [ ]:
#import cycle history, for each building, filings safe or unsafe

import pandas as pd
import numpy as np

df = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Cycle 5-8 Non DOB NOW Facade Filings (11-22-16).csv", low_memory=False )

df['Initial File Date Raw'] = df['Initial File Date Raw'].astype(str)
df['Initial File Date'] =  df['Initial File Date Raw'].apply(lambda x: x[4:6]) + "/" + df['Initial File Date Raw'].apply(lambda x: x[6:8]) + "/" + df['Initial File Date Raw'].apply(lambda x: x[:4])

df['Initial File Date'] = df['Initial File Date'].str.replace('//', '')

df['Initial File Year'] = df['Initial File Date Raw'].apply(lambda x: x[:4])

#df["BIN"] = df["BIN"].astype(int)

print "drop current status of NR"
print "original length of df", len(df)
df = df[df['Current Status'] != 'NR']
#df = df[df['BIN'] != ' ']

df = df.reset_index(drop=True)
print "new length of df", len(df)

print
print "drop rows with BIN = 0"
df = df[np.isfinite(df['BIN'])]
print "new length of df", len(df)
df["BIN"] = df["BIN"].astype(int)

df = df[["BIN", "Facade Cycle", "Number of Stories", "Initial Filing Status", "Current Status", \
         "Initial File Date", "Initial File Year", "Owner Name", "Owner Business Name", "Owner Address", "Owner City", "Owner State", \
         "Owner Type", "Boro", "Block", "Lot"]]

#drop NaN

print 
print "drop NaN"
print "length of df before drop", len(df)
df = df.dropna()
print "length of df after drop", len(df)

df = df.reset_index(drop=True)

#drop data before 2005
print

df = df.reset_index(drop=True)


In [ ]:
df.head(3)

In [ ]:
#create bbl for df
df = df.reset_index(drop=True)

df["Boro"] = df["Boro"].astype(int)
df["Block"] = df["Block"].astype(int)
df["Lot"] = df["Lot"].astype(int)

df["Boro"] = df["Boro"].astype(str)
df["Block"] = df["Block"].astype(str)
df["Lot"] = df["Lot"].astype(str)

df["Boro"] = df["Boro"].map(str.strip)
df["Block"] = df["Block"].map(str.strip)
df["Lot"] = df["Lot"].map(str.strip)



#create BBL
for i in range(0, len(df)):
    if len(df["Block"][i]) == 1:
        df["Block"][i] = "0000" + df["Block"][i] 
    if len(df["Block"][i]) == 2:
        df["Block"][i] = "000" + df["Block"][i]
    if len(df["Block"][i]) == 3:
        df["Block"][i] = "00" + df["Block"][i]
    if len(df["Block"][i]) == 4:
        df["Block"][i] = "0" + df["Block"][i]
 
    if len(df["Lot"][i]) == 1:
        df["Lot"][i] = "000" + df["Lot"][i]
    if len(df["Lot"][i]) == 2:
        df["Lot"][i] = "00" + df["Lot"][i]
    if len(df["Lot"][i]) == 3:
        df["Lot"][i] = "0" + df["Lot"][i] 

        
df["BBL"] = df["Boro"] +  df["Block"] + df["Lot"]

In [ ]:
#import tax liens to add to dataset
dfTax =  pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Tax_Lien_Sale_Lists.csv", low_memory=False )

dfTax = dfTax.rename(columns={'Block ': 'Block'})


dfTax["Borough"] = dfTax["Borough"].astype(int)
dfTax["Block"] = dfTax["Block"].astype(int)
dfTax["Lot"] = dfTax["Lot"].astype(int)

dfTax["Borough"] = dfTax["Borough"].astype(str)
dfTax["Block"] = dfTax["Block"].astype(str)
dfTax["Lot"] = dfTax["Lot"].astype(str)

dfTax["Borough"] = dfTax["Borough"].map(str.strip)
dfTax["Block"] = dfTax["Block"].map(str.strip)
dfTax["Lot"] = dfTax["Lot"].map(str.strip)


#create BBL
for i in range(0, len(dfTax)):
    if len(dfTax["Block"][i]) == 2:
        dfTax["Block"][i] = "000" + dfTax["Block"][i]
    if len(dfTax["Block"][i]) == 3:
        dfTax["Block"][i] = "00" + dfTax["Block"][i]
    if len(dfTax["Block"][i]) == 4:
        dfTax["Block"][i] = "0" + dfTax["Block"][i]
 
    if len(dfTax["Lot"][i]) == 1:
        dfTax["Lot"][i] = "000" + dfTax["Lot"][i]
    if len(dfTax["Lot"][i]) == 2:
        dfTax["Lot"][i] = "00" + dfTax["Lot"][i]
    if len(dfTax["Lot"][i]) == 3:
        dfTax["Lot"][i] = "0" + dfTax["Lot"][i] 

        
dfTax["BBL"] = dfTax["Borough"] +  dfTax["Block"] + dfTax["Lot"]

#get set of bbls 
tax_list = []

for i in range(0, len(dfTax)):
    tax_list.append(dfTax["BBL"][i])
    
tax_set = set(tax_list)
tax_set = list(tax_set)

print "length of dfTax", len(dfTax)
print "length of tax_set", len(tax_set)

In [ ]:
#label if have tax lien
df["Tax Lien"] = 0
count = 0
for i in range(0, len(df)):
    if df["BBL"][i] in tax_set:
        count+=1
        df["Tax Lien"][i] = 1
print count

In [ ]:
df["BBL"][0]

In [ ]:
print len(df)

In [ ]:
#assign a pluto owner to each bin
df["BIN"] = df["BIN"].astype(str)

df["PlutoOwner"] = ''

count=0



for i in range(0, len(df)):
    if df["BIN"][i] in dfOwn:
        count+=1
        df["PlutoOwner"][i] = dfOwn[df["BIN"][i]]
print count

print "length of df before dropping null owners", len(df)
df = df[pd.notnull(df['PlutoOwner'])]
#df = df[df["PlutoOwner"] != 'none']
print "length after drop", len(df)
df = df.reset_index(drop=True)

In [ ]:
#Create unsafe, safe and swarmp fields
pd.set_option('chained_assignment', None)

df["Initial Filing Status"] = df["Initial Filing Status"].map(str.strip)

df["UN"] = 0
df["SW"] = 0
df["OK"] = 0

for i in range(0, len(df)):
    if df["Initial Filing Status"][i] == "UN":
        df["UN"][i] = 1
    if df["Initial Filing Status"][i] == "SW":
        df["SW"][i] = 1
    if df["Initial Filing Status"][i] == "OK":
        df["OK"][i] = 1
    
    

In [ ]:
df.head()

In [ ]:
#count up the number of unsafe, safe and swarmp initial status
df2 = df[["PlutoOwner", 'UN', 'SW', 'OK', 'Tax Lien']]

#clean address first before grouping
df2["PlutoOwner"] = df2["PlutoOwner"].astype(str)
df2["PlutoOwner"] = df2["PlutoOwner"].str.replace('[^\x00-\x7F]','')
df2["PlutoOwner"] = df2["PlutoOwner"].str.replace('.',' ')
df2["PlutoOwner"] = df2["PlutoOwner"].str.replace(',',' ')
df2["PlutoOwner"] = df2["PlutoOwner"].str.replace('  ',' ')
df2["PlutoOwner"] = df2["PlutoOwner"].map(str.strip)

df2G = (df2.groupby('PlutoOwner').agg({'UN':'sum', 'SW': 'sum', 'OK':'sum', 'Tax Lien': 'sum'})).reset_index()


df2G["Risk1"] = (0.7*df2G["UN"] + 0.3*df2G["SW"] + 0*df2G["OK"])/(df2G["UN"] + df2G["SW"] + df2G["OK"])

df2G["Risk Unsafe"] = (df2G["UN"])/(df2G["UN"] + df2G["SW"] + df2G["OK"])

df2G = df2G.sort_values(by = 'Risk Unsafe', ascending=False).reset_index(drop=True)

print len(df2G)
print len(df2)




In [ ]:
df2G.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\OWNERSHIP RISK.csv")

In [ ]:
df2G = df2G[["PlutoOwner", "UN", "SW", "OK","Tax Lien", "Risk1", "Risk Unsafe"]]
df2G

In [ ]:
"""
#use pluto data to determine if we know the filing history
dfP = dfPluto[["bin", "OwnerName", "Address", "NumFloors",  "Borough", "XCoord", "YCoord"]]

dfP["OwnerName"] = dfP["OwnerName"].astype(str)
dfP["OwnerName"] = dfP["OwnerName"].str.replace('[^\x00-\x7F]','')
dfP["OwnerName"] = dfP["OwnerName"].str.replace('.',' ')
dfP["OwnerName"] = dfP["OwnerName"].str.replace(',',' ')
dfP["OwnerName"] = dfP["OwnerName"].str.replace('  ',' ')
dfP["OwnerName"] = dfP["OwnerName"].map(str.strip)

dfP = dfP[dfP["NumFloors"] >= 6]

dfP = dfP.reset_index(drop=True)


print len(dfP)
"""

In [ ]:
#label each bin as known or unknown filing history, known if bin is in the 
#APPLY TO SUBSET OF PLUTO DATA
df = df.reset_index(drop=True)

#get set of bins
bin_list = []

for i in range(0, len(df)):
    bin_list.append(df["BIN"][i])
    
bin_set = set(bin_list)
bin_set = list(bin_set)

dfP["Compliance Known"] = ""

for i in range(0, len(dfP)):
    if i%1000 == 0:
        print i
    if dfP["bin"][i] in bin_set:
        dfP["Compliance Known"][i] = 1
    else:
        dfP["Compliance Known"][i] = 0
        
        
count = dfP[dfP["Compliance Known"] == 1]
print
print len(count)

In [ ]:
#label each bin as known or unknown filing history, known if bin is in the 
#APPLY TO ALL OF PLUTO DATA
df = df.reset_index(drop=True)


#Add failure to maintain and complaints
#Add Facade violations
fv = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Failure to maintain_protect (All years, all classes, REVISED).csv", low_memory=False )
fv["Vio Issued Date"] = pd.to_datetime(fv["Vio Issued Date"])

fv["BIN"] = fv["BIN"].astype(str)

initial_date = "1-1-2006"
#print "initial date", initial_date
mask = (fv['Vio Issued Date'] > initial_date)
fv = fv.loc[mask] 

fv = fv.sort_values(by = 'Vio Issued Date', ascending=True).reset_index(drop=True)

#get set of bins for failure to maintain
f_list = []
for i in range(0, len(fv)):
    f_list.append(fv["BIN"][i])
    
f_set = set(f_list)
f_set = list(f_set)


dc = pd.read_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\Facade, Brickwork, Exterior Wall Complaints.csv", low_memory=False )

dc["Date Entered"] = pd.to_datetime(dc["Date Entered"])
dc["BIN Number"] = dc["BIN Number"].astype(str)

initial_date = "1-1-2006"
#print "initial date", initial_date
mask = (dc["Date Entered"] > initial_date)
dc = dc.loc[mask] 

dc = dc.sort_values(by = 'Date Entered', ascending=True).reset_index(drop=True)

#get set of bins for complaints
c_list = []
for i in range(0, len(dc)):
    c_list.append(dc["BIN Number"][i])
    
c_set = set(c_list)
c_set = list(c_set)



#get set of bins
bin_list = []

for i in range(0, len(df)):
    bin_list.append(df["BIN"][i])
    
bin_set = set(bin_list)
bin_set = list(bin_set)



dfPluto["Compliance Known"] = 0
dfPluto["Failure Maintain"] = 0
dfPluto["Complaint"] = 0

for i in range(0, len(dfPluto)):
    if i%50000 == 0:
        print i
    if dfPluto["bin"][i] in bin_set:
        dfPluto["Compliance Known"][i] = 1
    if dfPluto["bin"][i] in f_set:
        dfPluto["Failure Maintain"][i] = 1
    if dfPluto["bin"][i] in c_set:
        dfPluto["Complaint"][i] = 1
        
        
count = dfPluto[dfPluto["Compliance Known"] == 1]
print
print len(count)

In [ ]:
dfPluto["bin"][0]

In [ ]:
f_set[0]

In [ ]:
dfPluto.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\PLUTO\\MapPLUTO_BINs_1toManyJOINCompKnown.csv", low_memory=False )


In [ ]:
#subset the cycle history data to get only very bad actors
bad_actors1 = df2G[df2G["Risk Unsafe"] > 0.65]
bad_actors1 = bad_actors1[bad_actors1["UN"] > 2]
bad_actors1 = bad_actors1.reset_index(drop=True)

print len(df2G)
print len(bad_actors1)

In [ ]:
bad_actors1

In [ ]:
#get bad actor owner list
#get set of bins
bad_list = []

for i in range(0, len(bad_actors1)):
    bad_list.append(bad_actors1["PlutoOwner"][i])
    
bad_set = set(bad_list)
bad_set = list(bad_set)


In [ ]:
#get bad actors

qn = pd.DataFrame(columns=('BIN', 'OwnerName', 'Address', 'Boro', 'NumFloors', 'XCoord', 'YCoord','Compliance Known', 'Failure Maintain', 'Facade Complaint', 'YearBuilt'))

dfPluto = dfPluto.reset_index(drop=True)
   
        
for i in range(0, len(dfPluto)):
    if i%50000 == 0:
        print i
    if dfPluto["OwnerName"][i] in bad_set:
        qn.loc[i] =  [dfPluto["bin"][i], dfPluto["OwnerName"][i], dfPluto["Address"][i], dfPluto["Borough"][i], dfPluto["NumFloors"][i],dfPluto["XCoord"][i], dfPluto["YCoord"][i], dfPluto["Compliance Known"][i], \
                      dfPluto["Failure Maintain"][i], dfPluto["Complaint"][i], dfPluto["YearBuilt"][i]   ]

In [ ]:
qn

In [ ]:
"""
qn["Address"] = qn["Address"].map(str.strip)
qn["Boro"] = qn["Boro"].map(str.strip)
qn["Address Full"] = qn["Address"] + " " + qn["Boro"]

qn2 = (qn.groupby('Address Full').agg({'Compliance Known':'sum', 'Failure Maintain':'sum', 'Facade Complaint': 'sum'})).reset_index()

qn2 = qn2.sort_values(by = 'Compliance Known', ascending=True).reset_index(drop=True)
"""

In [ ]:
bad_set[0]

In [ ]:
qn["Address"] = qn["Address"].map(str.strip)
qn["Boro"] = qn["Boro"].map(str.strip)
qn["Address Full"] = qn["Address"] + " " + qn["Boro"]

qn = qn.reset_index(drop=True)

#get owner name list

counter = 0

final_list = pd.DataFrame(columns=('BIN', 'OwnerName', 'Address',  'NumFloors', 'XCoord', 'YCoord', 'Facade Complaint', 'YearBuilt'))

for i in range(0, len(bad_set)):
    #subset by owner
    qn2 = qn[qn['OwnerName'] == bad_set[i]]
    qn2 = qn2.reset_index(drop=True)
    
    for j in range(0, len(qn2)):
        #subset by address
        qn3 = qn2[qn2['Address Full'] == qn2["Address Full"][j]]
        
        #sum compliance and failure to maintain, if zero then building is not known to DOB
        sum_comp = qn3["Compliance Known"].sum()
        sum_failure = qn3["Failure Maintain"].sum()
        #sum_complaint = qn3["Facade Complaint"].sum()
        
        if (sum_comp == 0 and sum_failure == 0):
            #print qn3["BIN"][j],qn3["OwnerName"][j],qn3["Address Full"][j]  
            print qn3["BIN"][j],qn3["OwnerName"][j],qn3["Address Full"][j],qn3["NumFloors"][j], qn3["XCoord"][j],qn3["YCoord"][j], qn3["Facade Complaint"][j] 
            final_list.loc[counter] = [qn3["BIN"][j],qn3["OwnerName"][j],qn3["Address Full"][j],qn3["NumFloors"][j], qn3["XCoord"][j],qn3["YCoord"][j], qn3["Facade Complaint"][j], qn3["YearBuilt"][j]  ]
            counter +=1



    
    
    #print qn2['OwnerName'],  qn2['Address']
    #print
    

In [ ]:
final_list

final_list = final_list[final_list["NumFloors"] > 2]
final_list = final_list.drop_duplicates(subset=['Address']).reset_index(drop=True)

print len(final_list)

final_list.to_csv("S:\\Office of Risk Management\\User Folders\\Darius Mehri\\Facade Predictive Analysis\\final_list2.csv", low_memory=False )


In [ ]:
final_list